In [1]:
!nvidia-smi

Sat May  8 14:53:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K40m          Off  | 00000000:81:00.0 Off |                    0 |
| N/A   60C    P0   119W / 235W |   5616MiB / 11441MiB |     94%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
%cd /home/mtech1/19CS60R28/susmit/DocRed_hongwang600/DocRed
import sys
import nltk
import numpy as np
import os
import json

from nltk.tokenize import WordPunctTokenizer
from pytorch_transformers import *
from models.bert import Bert
import seaborn as sns
%cd ..

%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

/home/mtech1/19CS60R28/susmit/DocRed_hongwang600/DocRed
/home/mtech1/19CS60R28/susmit/DocRed_hongwang600


# Data

In [3]:
in_path='data'
out_path='prepro_data'

In [4]:
train_annotated_file_name = os.path.join(in_path, 'train_annotated.json')
dev_file_name = os.path.join(in_path, 'dev.json')
test_file_name = os.path.join(in_path, 'test.json')

In [5]:
rel2id = json.load(open(os.path.join(out_path, 'rel2id.json'), "r"))
id2rel = {v:u for u,v in rel2id.items()}
json.dump(id2rel, open(os.path.join(out_path, 'id2rel.json'), "w"))

In [7]:
BATCH_SIZE=8
bert = Bert(BertModel, 'bert-base-uncased')

# Preprocessing

In [7]:
list=[]
ori_data=json.load(open(train_annotated_file_name))
for x in ori_data:
    s=0
    for y in x['sents']:
        s+=len(y) 
    list.append(s)
list[:10]
import sns 

[40, 31, 16, 11, 21, 45, 22, 11, 17, 41]

In [9]:
# def preprocess(data_file_name, max_length = 512, is_training = True, suffix=''):
#     ori_data=json.load(open(data_file_name))
a,b,c=bert.subword_tokenize_to_ids(['This','is', 'my','name','.','Today','we','are','going','to'])
print(a)
print(b)
print(c)


[[ 101 2023 2003 2026 2171 1012 2651 2057 2024 2183 2000  102    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [8]:
init(train_annotated_file_name, rel2id, max_length = 512, is_training = False, suffix='_train')

data_len: 50
Saving files
Finishing processing
Finish saving


In [40]:
ori_data=json.load(open(train_annotated_file_name))
list=[]
for x in ori_data:
    dict={}
    for y in x['labels']:
        for z in y['evidence']:
            if (y['h'],y['t']) not in dict:
                dict[(y['h'],y['t'])]=[0]*len(x['sents'])
            dict[(y['h'],y['t'])][z]=1
    for key in dict:
        list.append(dict[key])
s=0
for x in list:
    s+=sum(x)/len(x)
print(s/len(list))

0.22515528361237225


In [24]:
Dictionary1 = { 'A': 'Geeks', 'B': 'For', 'C': 'Geeks' }
  
# Dictionary to be checked
print("Dictionary to be checked: ")
print(Dictionary1)
  
# Use of has_key() to check
# for presence of a key in Dictionary
print(Dictionary1.has_key('A'))
print(Dictionary1.has_key('For'))

Dictionary to be checked: 
{'A': 'Geeks', 'B': 'For', 'C': 'Geeks'}


AttributeError: 'dict' object has no attribute 'has_key'

In [6]:
ori_data=json.load(open(train_annotated_file_name))
list=[]
for x in ori_data:
    

In [45]:
def preprocess(data_file_name,max_length=512,is_training=True,suffix=''):
    ori_data = json.load(open(data_file_name))
    sent_ids=[]
    for x in ori_data:
        for sent in x['sents']:
            sent,_,_=bert.subword_tokenize_to_ids(sent)
            sent=np.reshape(sent,(max_length))
            sent_ids.append(sent)

    bert_token = np.asarray(sent_ids,dtype = np.int64)
    label=np.zeros(bert_token.shape[0])
    s=0
    for i in range(len(ori_data)):
        for j in range(len(ori_data[i]['labels'])):
            for x in ori_data[i]['labels'][j]['evidence']:
                label[x+s]=1
        s+=len(ori_data[i]['sents'])
    print("Total sentences = ",s)
    print("Saving files")
    np.save(os.path.join(out_path,suffix+'bert_token.npy'),bert_token)
    np.save(os.path.join(out_path,suffix+'label.npy'),label)
    print("Saving files compelted")
    
    

preprocess(train_annotated_file_name,max_length=512,is_training=True,suffix='_train')
preprocess(dev_file_name,max_length=512,is_training=False,suffix='_dev')




24256 total sent count
Saving files
[1. 0. 1. ... 1. 1. 1.]
Saving files compelted


In [19]:

bert_token=np.load(os.path.join(out_path,"train"+"_train"+'bert_token.npy'))

In [46]:
label=np.load(os.path.join(out_path,"train"+"_train"+'label.npy'))

In [48]:
print(label.shape[0],sum(label))

24256 13033.0


In [5]:
ori_data=json.load(open(train_annotated_file_name))
print(ori_data[0])

{'vertexSet': [[{'pos': [0, 4], 'type': 'ORG', 'sent_id': 0, 'name': 'Zest Airways, Inc.'}, {'sent_id': 0, 'type': 'ORG', 'pos': [10, 15], 'name': 'Asian Spirit and Zest Air'}, {'name': 'AirAsia Zest', 'pos': [6, 8], 'sent_id': 0, 'type': 'ORG'}, {'name': 'AirAsia Zest', 'pos': [19, 21], 'sent_id': 6, 'type': 'ORG'}], [{'name': 'Ninoy Aquino International Airport', 'pos': [4, 8], 'sent_id': 3, 'type': 'LOC'}, {'name': 'Ninoy Aquino International Airport', 'pos': [26, 30], 'sent_id': 0, 'type': 'LOC'}], [{'name': 'Pasay City', 'pos': [31, 33], 'sent_id': 0, 'type': 'LOC'}], [{'name': 'Metro Manila', 'pos': [34, 36], 'sent_id': 0, 'type': 'LOC'}], [{'name': 'Philippines', 'pos': [38, 39], 'sent_id': 0, 'type': 'LOC'}, {'name': 'Philippines', 'pos': [13, 14], 'sent_id': 4, 'type': 'LOC'}, {'sent_id': 5, 'type': 'LOC', 'pos': [25, 29], 'name': 'Republic of the Philippines'}], [{'name': 'Manila', 'pos': [13, 14], 'sent_id': 1, 'type': 'LOC'}, {'name': 'Manila', 'pos': [9, 10], 'sent_id': 3,

In [53]:
bert.subword_tokenize_to_ids(['hi','how','are','you'])


(array([[ 101, 7632, 2129, 2024, 2017,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0

In [54]:
a.add(('a','b','c'))

NameError: name 'a' is not defined

7=▽  ]11;?"models/bert.py" 117L, 4364C[>c        ----------
        tokens: A sequence of strings, representing input tokens.

        Returns
        -------
        A tuple consisting of:
            - A list of subword IDs, including IDs of the special
                symbols (CLS and SEP) required by Bert.
            - A mask indicating padding tokens.
            - An array of indices into the list of subwords. See
                doc of subword_tokenize.
        """subwords, token_start_idxs = self.subword_tokenize(tokens)subword_ids, mask = self.convert_tokens_to_ids(subwords)token_starts = torch.zeros(1, self.max_len).to(subword_ids)print(token_starts)token_starts[0, token_start_idxs] = 1return subword_ids.numpy(), mask.numpy(), token_starts.numpy()def segment_ids(self, segment1_len, segment2_len):ids = [0] * segment1_len + [1] * segment2_lenreturn torch.tensor([ids])117,0-1Bot
Type  :quit<Enter>  to exit Vim117,0-1Bot

config/         gen_data.py*  README.md*         test.py*     train.py*
evaluation.py*  models/       requirements.txt*  test_sp.py*  train_sp.py*


RuntimeError: CUDA error: no kernel image is available for execution on the device